# Discret logarithmic problème

In [1]:
import sympy as sp
import numpy as np
import math

## Dans le groupe additif $( \mathbb{Z}_p, + )$

étant donné un nombre premier \(p\), un générateur \(g\) du groupe additif $\mathbb{Z}_p$, et un élément $y \in \mathbb{Z}_p$, trouver $x$ tel que :
    $a \cdot x \equiv b \pmod{p}$

In [2]:
# variables globales
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_model_map={}
x=sp.Add()
k=sp.Add()

In [3]:
# Créer un symbole à partir d'un nom en string
def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]

    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1
    else:
        x_4 = key

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

def transfers_qubo2Ising(reduct_demension_expr):
    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

# Compose the sampler
base_sampler = dimod.ExactSolver()
composed_sampler = SpinReversalTransformComposite(base_sampler)

def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

def get_solution(expr_ising):
    (linear,quadratic,offset) = get_solver_parameter(expr_ising)
    response = composed_sampler.sample_ising(linear, quadratic)  #response.data() return (solution, energy, num_occurrences)
    solution = next(response.data())[0]
    return solution

In [4]:
# set variable en binaire qui est dans {0,...,p-1}

# exemple : x = u1 + 2u2 + 4u3 + ... + 2^(n-2)un-1 + (p-2^(n-1)+1)un
def set_x_expression(n,p):  # n est length de x en binaire
    # len(x) = len(p)
    expr_x = 0
    # de u1 à un-1
    for i in range(1,n):
        var_name = ('u' + str(i))
        var = create_variables(var_name)
        expr_x = 2**(i-1) * var + expr_x
    # un
    expr_x = expr_x + (p-2**(n-1)+1) * create_variables('u' + str(n))
    return expr_x

def set_k_expression(n):  # l est length de k en binaire
    l = math.floor(math.log2(n)) + 1
    expr_k = 0
    # de u1 à un-1
    for i in range(1,l):
        var_name = ('k' + str(i))
        var = create_variables(var_name)
        expr_k = 2**(i-1) * var + expr_k
    # un
    expr_k = expr_k + (n-2**(l-1)+1) * create_variables('k' + str(l))
    return expr_k

Ici on donne la taille de x :n = 3 comme parapmètre, x et p ont la même taille.

In [5]:
x = set_x_expression(3,5)
x

u1 + 2*u2 + 2*u3

### définir fonction
$(x \cdot a \bmod p ) + (-b \bmod p) - k\cdot p = 0 $

In [6]:
def define_function(a, b, p):
    # set variables for x
    n = math.ceil(math.log2(p))  # len de x == len de p
    global x
    x = set_x_expression(n,p)

    #set variables for k
    global k
    k = set_k_expression(n)

    # set expression
    expr = x * sp.Mod(a,p)
    expr = expr + sp.Mod(-b,p)
    expr = expr - (k * p)
    expr = (expr) **2
    expr = sp.expand(expr)
    return expr

In [7]:
expr = define_function(2,3,5)
expr

25*k1**2 + 100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 - 20*k1 + 100*k2**2 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 - 40*k2 + 4*u1**2 + 16*u1*u2 + 16*u1*u3 + 8*u1 + 16*u2**2 + 32*u2*u3 + 16*u2 + 16*u3**2 + 16*u3 + 4

In [8]:
expr_reduct_power = reduct_variable_power_function(expr)
expr_reduct_power

100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 + 60*k2 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4

In [9]:
# obtient l'expression finale
def get_reduct_expression(a,b,p):
    expr = define_function(a,b,p)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
    return reduct_demension_expr

In [10]:
reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
reduct_demension_expr

100*k1*k2 - 20*k1*u1 - 40*k1*u2 - 40*k1*u3 + 5*k1 - 40*k2*u1 - 80*k2*u2 - 80*k2*u3 + 60*k2 + 16*u1*u2 + 16*u1*u3 + 12*u1 + 32*u2*u3 + 32*u2 + 32*u3 + 4

## On obtient l'expression sous forme de QUBO, ensuit on peut la transferer sous forme de modèle d'Ising

In [11]:
expr_ising = transfers_qubo2Ising(reduct_demension_expr)
expr_ising

25*k1*k2 - 5*k1*u1 - 10*k1*u2 - 10*k1*u3 + 5*k1/2 - 10*k2*u1 - 20*k2*u2 - 20*k2*u3 + 5*k2 + 4*u1*u2 + 4*u1*u3 - u1 + 8*u2*u3 - 2*u2 - 2*u3 + 81/2

# Find solution avec dimon bibilothèque

In [12]:
solution = get_solution(expr_ising)
solution

{'k2': np.int8(1),
 'u2': np.int8(1),
 'u3': np.int8(1),
 'k1': np.int8(-1),
 'u1': np.int8(-1)}

In [13]:
def calcule_x(solution,x,k):
    dic_varialbe_value = {}
    for var in variables:
        if(solution[var.name] == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)

    x_value = x.subs(dic_varialbe_value)
    k_value = k.subs(dic_varialbe_value)
    
    return (x_value.evalf(),k_value.evalf())


def factorization(a, b, p):
    global variables, variables_Ising, dictionaire_auxiliary_variable, res,x,k
    variables = []
    variables_Ising = []
    dictionaire_auxiliary_variable = {}
    res = 1
    x= sp.Add()
    k= sp.Add()


    reduct_demension_expr = get_reduct_expression(a,b,p)
    expr_ising = transfers_qubo2Ising(reduct_demension_expr)
    solution = get_solution(expr_ising)
    (x_val,k_val) = calcule_x(solution,x,k)
    x_int = int(x_val)
    k_int = int(k_val)
    print(f" k: {k_int}")
    print(f"The solution of {a} * x = {b} mod {p} is : x = {x_int}.")

In [14]:
a=2
b=3
p=5
factorization(a, b, p)

 k: 2
The solution of 2 * x = 3 mod 5 is : x = 4.


In [15]:
a=5
b=5
p=7
factorization(a, b, p)

 k: 1
The solution of 5 * x = 5 mod 7 is : x = 1.


# Dans le groupe multiplicative $( \mathbb{Z}_p, \cdot )$

## Brutal approach
On define le DLP comme ceci:
    $g^y = h $

Ensuite, dans $\mathbb{F}_p, g,h \in \{1, \cdots , p-1\}, y \in \{1, \cdots, \text{Ord}(g)\}$


$g^y \equiv h \pmod{p}$


#### Defint y:
set $m$ est le nombre de bits de $y$ = bitlength of ord(g).

$y$ peux être définit comme ceci:
    $ y = 2^{m - 1} u_m + \cdots + 2u_2 + u_1$

union de equation 6 et 8, on obtient equation 13

## Fonctions basiques:
1. calculer ord(g) dans p
2. définir valeur unconnu en l'expression composé des variables binaires

In [16]:
def calculate_order(g, p):
    # ord(g): le min k qui satisfait g^k = 1 mod p
    if g <= 0 or g >= p:
        print(f"error：g should in { 1, p-1} ")
        return None

    order = 1
    current = g % p

    while current != 1:
        # g^(order+1) mod p
        current = (current * g) % p
        order += 1

        if order > p-1:
            print(f"error：p is not prime or g and p are not prime")
            return None
    return order

def set_variable_expression(n,prefix):  # n is the length of variable and the prefix of each binary variable
    expr_var = 0
    # de u1 à un
    for i in range(1,n+1):
        var_name = (prefix + str(i))
        var = create_variables(var_name)
        expr_var = 2**(i-1) * var + expr_var
    return expr_var

In [17]:
g= 2
p= 5
h= 3
ord = calculate_order(g,p)
n = math.ceil(math.log2(ord))
expr_y = set_variable_expression(n,'u')
print(f"expression of y: {expr_y}")
expr_k = set_variable_expression(n,'k')
print(f"expression of k: {expr_k}")

expression of y: u1 + 2*u2
expression of k: k1 + 2*k2


## Definir $g^y$ en forme de equation 6:

$g^y = g^{2^{m-1} \cdot u_m+\cdots+2u_2+u_1}  = g^{2^{m-1}} \cdots g^{2u_2}g^{u_1}$

In [18]:
def create_g_power_expression(expr_y,g):
    """
    g^y = g^(2^0*u₁ + 2^1*u₂+ ... + 2^(n-1)*uₙ)
    """
    #global g_symbol
    #g_symbol = sp.Symbol('g')

    g_power_y = g ** expr_y

    return g_power_y

def decompose_g_power(g_power_expr):
    """
    g^y = g^(2^0*u₁) * g^(2^1*u₂) * ... * g^(2^(n-1)*uₙ)
    """
    exponent = g_power_expr.as_base_exp()[1]
    base = g_power_expr.as_base_exp()[0]
    decomposed_expr = 1

    # u₁ + u₂·2¹ + ... + uₙ·2ⁿ⁻¹
    for term in exponent.args:
        decomposed_expr = decomposed_expr * base**term
    return decomposed_expr

In [19]:
g_power_y = create_g_power_expression(expr_y,g)
print(f"expression of g^y : {g_power_y}")

decomposed_expr = decompose_g_power(g_power_y)
print(f"expression of decomposed g^y : {decomposed_expr}")

expression of g^y : 2**(u1 + 2*u2)
expression of decomposed g^y : 2**u1*2**(2*u2)


## Transforme en utilisant l'équation 8 de l'equation 6
transfert formule : $g^{2^{i-1} \cdot u_i} = 1+u_i(g^{2^{i-1}}-1)$

$g^y =(1 + u_m(g^{2^{m−1}}− 1))  \cdots (1 + u_2 (g^2 − 1)) \dot (1 + u_1 (g − 1))$


In [20]:
def transform_using_formula(decomposed_expr):
    """
    g^(2^(i-1) * u_i) = 1 + u_i * (g^(2^(i-1)) - 1 % p)   #equation 13
    """
    terms = []
    for element in decomposed_expr.args:
        ele = element.exp

        if ele.is_symbol:
            #i = 1
            transformed_element = 1 + ele * (element.base - 1)
            terms.append(transformed_element)
        elif ele.is_Mul:
            [coeff, u_var] = ele.args
            transformed_element = 1 + u_var * (element.base**(coeff) - 1)
            terms.append(transformed_element)
    return sp.Mul(*terms)

In [21]:
transformed_expr = transform_using_formula(decomposed_expr)
print(f"expression of transformed g^y: {transformed_expr}")

expression of transformed g^y: (u1 + 1)*(3*u2 + 1)


## Définir la fonction objective
$ (( f-h ) \bmod p - (k \cdot p))^2 = 0 $

avec $f =(1 + u_m(g^{2^{m−1}}− 1))  \cdots (1 + u_2 (g^2 − 1)) \dot (1 + u_1 (g − 1))$

In [24]:
def define_function(g, h, p):
    # set variables for
    ord = calculate_order(g,p)
    if ord :
        n = math.ceil(math.log2(ord))
    else:
        print(f"ord({g}) is none ")
        return None

    global expr_y
    expr_y = set_variable_expression(n,'u')
    print(f"expression of y: {expr_y}")

    # g^y = transformed_expr
    g_power_expr = create_g_power_expression(expr_y,g)
    decomposed_expr = decompose_g_power(g_power_expr)
    transformed_expr = transform_using_formula(decomposed_expr)
    print(f"expression of transformed g^y: {transformed_expr}")

    #set variables for k
    global expr_k
    expr_k = set_variable_expression(n,'k')   # contains variables : ki
    print(f"expression of k: {expr_k}")

    # set expression(equation 15)
    print("+++++++++++++++++++++")
    expr = transformed_expr - h
    expr = sp.expand(expr)

    expr = expr % p
    expr = sp.expand(expr)

    expr = expr - (expr_k * p)
    expr = sp.expand(expr)

    expr = (expr) **2
    expr = sp.expand(expr)
    return expr

In [25]:
# variables globales
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_model_map={}
y=sp.Add()
k=sp.Add()

reduct_demension_expr = get_reduct_expression(2,3,5)
print(f"reducted demension expression : {reduct_demension_expr}")
expr_ising = transfers_qubo2Ising(reduct_demension_expr)
print(f"ising expression : {expr_ising}")
#get_solution(expr_ising)
solution = get_solution(expr_ising)
solution

expression of y: u1 + 2*u2
expression of transformed g^y: (u1 + 1)*(3*u2 + 1)
expression of k: k1 + 2*k2
+++++++++++++++++++++
reducted demension expression : 100*k1*k2 - 20*k1*u1 + 40*k1*x2 + 25*k1 - 40*k2*u1 + 80*k2*x1 + 100*k2 + 80*u1*x1 + 40*u1*x2 - 20*u2*x1 - 10*u2*x2 - 120*x1 - 60*x2 + Mod(3*u1*u2 + u1 + 3*u2 + 3, 5)
ising expression : 25*k1*k2 - 5*k1*u1 + 10*k1*x2 + 85*k1/2 - 10*k2*u1 + 20*k2*x1 + 85*k2 + 20*u1*x1 + 10*u1*x2 + 15*u1 - 5*u2*x1 - 5*u2*x2/2 - 15*u2/2 - 25*x1 - 25*x2/2 + Mod(3*u1*u2/4 + 5*u1/4 + 9*u2/4 + 3/4, 5) + 35


{'k2': np.int8(-1),
 'u1': np.int8(-1),
 'k1': np.int8(-1),
 'u2': np.int8(1),
 'x1': np.int8(1),
 'x2': np.int8(1)}

In [26]:
def calcule_y(solution,expr_y,expr_k):
    dic_varialbe_value = {}
    for var in variables:
        if(solution[var.name] == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)

    y_value = expr_y.subs(dic_varialbe_value)
    k_value = expr_k.subs(dic_varialbe_value)

    return (y_value.evalf(),k_value.evalf())

calcule_y(solution,expr_y,expr_k)

(2.00000000000000, 0)